In [16]:
import numpy as np
import pandas as pd
import json
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pickle as pkl
import imblearn
np.random.seed(1332)
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
#import seaborn as sns
import statsmodels.api as sm
%matplotlib inline
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import sys                                               #### Filling the missing values using KNN because the dataset is very less
from impyute.imputation.cs import fast_knn
import imblearn                                      ### sampling the data using the smote method
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
import sklearn
import xgboost
from xgboost import XGBClassifier              ### XGBoost classification method
import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from xgboost import cv

In [17]:
excel = pd.ExcelFile('Coronna Data CERTAIN with KVB edits.xlsx')
df = pd.read_excel(excel, 'BL+3M')

In [18]:
df = df.drop(columns = ['SubjectID', 'CDate', 'Match'])    ### Unique values
df = df.drop(columns = ['SubjectID.1', 'UNMC_id.1', 'CDate.1', 'init_group.1', 'grp.1', 'UNMC_id.2', 'grp.2', 'init_group.2', 'CDate.1', 'futime.1'])    ### Duplicate values
df = df.drop(columns = ['init_group', 'futime'])    ### same values for all rows

In [19]:
df = df.drop(columns = ['statin_use', 'rfstatus_impute', 'ccpstatus_impute', 'statin_use.1', 'smkyrs', 'numcigs'])  ### Dropping columns who have null values greater than 70%

In [20]:
df = df.drop(columns = ['DAS28-CRP 3m', 'DAS28-CRP BL'])     ### deleting the regression based variables

In [21]:
df = df.rename(columns={"usresultsIgA.1": "usresultsIgA_BL", "usresultsIgG.1": "usresultsIgG_BL", "usresultsIgM.1": "usresultsIgM_BL", 'seatedbp1.1': 'seatedbp1_BL',
                              'seatedbp2.1': 'seatedbp2_BL', 'pres_mtx.1': 'pres_mtx_BL', 'pres_arava.1': 'pres_arava_BL', 'pres_azulfidine.1': 'pres_azulfidine_BL', 
                              'pres_plaquenil.1': 'pres_plaquenil_BL', 'pres_imuran.1': 'pres_imuran_BL', 'pres_minocin.1': 'pres_minocin_BL', 'pres_pred.1': 'pres_pred_BL',
                              'statin_use.1': 'statin_use_BL', 'tender_jts_28.1': 'tender_jts_28_BL', 'BLswollen_jts_28': 'swollen_jts_28_BL',
                              'BLmd_global_assess': 'md_global_assess_BL', 'BLpt_global_assess': 'pt_global_assess_BL', 'BLdi': 'di_BL', 'BLpt_pain': 'pt_pain_BL', 'BLusresultsCRP': 'usresultsCRP_BL',
                              'DAS28-CRP BL': 'DAS28-CRP_BL', })     ### Renaming some columns. Not sure why - did it because previous student did it

In [22]:
df = df.drop('UNMC_id',1)
final_df = df
model_label = LabelEncoder()
final_df['grp'] = model_label.fit_transform(final_df['grp'].astype('str'))
final_df['gender'] = model_label.fit_transform(final_df['gender'].astype('str'))
final_df['final_education'] = model_label.fit_transform(final_df['final_education'].astype('str'))
final_df['race_grp'] = model_label.fit_transform(final_df['race_grp'].astype('str'))
final_df['newsmoker'] = model_label.fit_transform(final_df['newsmoker'].astype('str'))
final_df['drinker'] = model_label.fit_transform(final_df['drinker'].astype('str'))
final_df['ara_func_class'] = model_label.fit_transform(final_df['ara_func_class'].astype('str'))

/state/partition1/job-15128777/ipykernel_640279/2061900728.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('UNMC_id',1)


In [23]:
final_df = final_df[final_df['3MResponse'] != 'Unknown']    ### We don't need unknown category in our classification data so removing that category


In [24]:
X = final_df.drop('3MResponse',1)                            ### Dividing the dataframe into X and y set
y = final_df['3MResponse']

/state/partition1/job-15128777/ipykernel_640279/2738362967.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = final_df.drop('3MResponse',1)                            ### Dividing the dataframe into X and y set


In [25]:
def KNN(new_df):
    sys.setrecursionlimit(100000) 
    imputed_training=fast_knn(new_df.values, k=15)
    return imputed_training
imputed_training = KNN(X)
c = list(X.columns)
df_without_nulls = pd.DataFrame(imputed_training, columns=c)

In [26]:
# lab = ['grp','gender','final_education','race_grp','newsmoker','drinker','ara_func_class']
# from scipy import stats
# for i in df_without_nulls.columns:
#     if i not in lab:
#         IQR = df_without_nulls[i].quantile(0.75) - df_without_nulls[i].quantile(0.25)
#         lower_limit = df_without_nulls[i].quantile(0.25) - 1.5 * IQR
#         upr_limit = df_without_nulls[i].quantile(0.75) + 1.5 * IQR
#         upr = df_without_nulls[i] > upr_limit
#         low = df_without_nulls[i] < lower_limit
#         #df_without_nulls = df_without_nulls[(df_without_nulls[i] < upr_limit) and (df_without_nulls[i] > lower_limit)].all()
#         df_without_nulls = df_without_nulls[~low | upr]

In [27]:
y= final_df.iloc[df_without_nulls.index,0]
y = model_label.fit_transform(y.astype('str'))

In [28]:
labels = ['grp','gender','final_education','race_grp','newsmoker','drinker','ara_func_class']     ### normalizing the dataset

for i in df_without_nulls.columns:
    if i not in labels:
        mean = df_without_nulls[i].mean()
        std = df_without_nulls[i].std()
        df_without_nulls[i] = (df_without_nulls[i] - mean) / std

In [29]:
labels = ['pres_imuran','pres_minocin','num_tnf','num_nontnf','hxunstab_ang','pres_minocin_BL','ethnicity','hxstroke','pres_imuran_BL']    #### This columns has same value for every row so when I divide by mean and std the value goes to infinity
for i in labels:
    df_without_nulls = df_without_nulls.drop(i,1)

/state/partition1/job-15128777/ipykernel_640279/1131187374.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_without_nulls = df_without_nulls.drop(i,1)


In [33]:
df_without_nulls['y'] = y

In [34]:
df_without_nulls

,grp,Type I IFN activity,IFNβ activity final,IFNα activity final,IFNβ/α ratio final,seatedbp1,seatedbp2,pres_mtx,pres_arava,pres_azulfidine,...,swollen_jts_28_BL,md_global_assess_BL,pt_global_assess_BL,di_BL,pt_pain_BL,usresultsCRP_BL,usresultsIgA_BL,usresultsIgG_BL,usresultsIgM_BL,y
0,0.0,-0.654306,-0.233305,-0.241645,0.335583,0.466451,1.515510,0.628933,-0.299786,-0.221089,...,-0.876955,-0.876463,-1.646836,-1.075567,-1.505828,-0.364306,0.360509,1.650275,-0.258713,0
1,1.0,-0.180156,-0.287982,-0.391540,0.045433,-0.840381,-0.418592,0.628933,-0.299786,-0.221089,...,-0.876955,-0.876463,-1.365084,-0.561403,-0.958866,-0.549165,-0.341491,-0.273531,-0.674561,2
2,1.0,2.011925,1.380809,1.973875,-0.508706,-0.365169,-0.633492,0.628933,-0.299786,-0.221089,...,-0.326631,-0.605368,-1.646836,-1.075567,-1.688149,-0.594519,-0.496478,-1.351473,-0.193053,1
3,1.0,-0.645107,-0.198797,-0.246166,0.363162,-0.365169,-0.633492,0.628933,-0.299786,-0.221089,...,0.223693,0.207917,-0.036826,-0.561403,0.317378,-0.560073,-1.827541,-0.462858,-0.729278,1
4,2.0,-0.256645,-0.389788,-0.474078,0.045433,0.110042,0.441009,0.628933,-0.299786,-0.221089,...,0.957459,0.750107,-1.244333,-0.561403,0.499698,-0.603130,-0.815568,-0.841512,-0.816825,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,2.0,-0.433723,-0.625475,-0.665157,0.045433,0.347648,0.226109,0.628933,-0.299786,-0.221089,...,0.223693,1.021202,-1.445585,-1.075567,0.499698,-0.271877,-1.089074,-0.304068,-0.444750,1
274,1.0,-0.654306,-0.165742,-0.047867,-0.144782,0.822859,-0.633492,-1.584275,-0.299786,-0.221089,...,-0.143190,0.207917,1.170682,0.981087,1.046660,-0.196670,0.989573,-0.734633,1.021663,1
275,2.0,-0.654306,0.409928,0.186820,0.136482,-0.008761,-1.063292,0.628933,-0.299786,-0.221089,...,-0.326631,-0.876463,-0.036826,-1.075567,-1.323508,0.666197,0.196405,0.703640,1.481285,2
276,2.0,-0.654306,0.184043,0.087093,0.108136,-0.365169,-1.922893,0.628933,-0.299786,-0.221089,...,-0.510073,-0.876463,-1.445585,-1.075567,-1.323508,-0.564666,-0.742633,1.222762,-0.805882,0


In [35]:
df_without_nulls.to_csv('preprocessing_approach_1.csv', index = False, )